<a href="https://colab.research.google.com/github/gabriel-picussa/MVP-de-SSD---Gabriel-Picussa/blob/main/MVP_Salary_CodeOnly_GHsafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# === MVP Salary — Code-Only Notebook (auto-run) ===
# This notebook loads data from GitHub, prepares it, performs EDA, trains/compares regression models,
# exposes an auto-updating prediction UI (ipywidgets), and shows an optional confusion matrix (didactic).

# ---------------------- Imports & Config ----------------------
GITHUB_URL = "https://raw.githubusercontent.com/gabriel-picussa/MVP-de-SSD---Gabriel-Picussa/main/Salary_DatA.xlsx"

try:
    import ipywidgets as widgets
except Exception:
    !pip -q install ipywidgets
    import ipywidgets as widgets

import pandas as pd, numpy as np, matplotlib.pyplot as plt, requests, io, re
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier

plt.rcParams['figure.figsize'] = (8,5)
pd.options.mode.copy_on_write = True

def to_raw_github_url(url: str) -> str:
    if "github.com" in url and "/blob/" in url:
        parts = url.split("github.com/")[-1]
        user_repo, rest = parts.split("/blob/", 1)
        return f"https://raw.githubusercontent.com/{user_repo}/{rest}"
    return url

# ---------------------- Load Data (auto) ----------------------
RAW = to_raw_github_url(GITHUB_URL)
print("Lendo:", RAW)
resp = requests.get(RAW, timeout=30)
resp.raise_for_status()
df = pd.read_excel(io.BytesIO(resp.content))
print("Dimensões:", df.shape)
display(df.head())

# ---------------------- Prep & Harmonization ----------------------
def prep_df(df):
    df = df.copy()
    df.columns = [str(c).strip().replace(' ', '_').replace('-', '_') for c in df.columns]
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    categorical_cols = [c for c in df.columns if c not in numeric_cols]
    df[numeric_cols] = df[numeric_cols].apply(lambda s: s.fillna(s.median()))
    for c in categorical_cols:
        df[c] = df[c].astype(str).fillna('Missing').replace({'nan':'Missing'})
    col = 'Education_Level' if 'Education_Level' in df.columns else ('education_level' if 'education_level' in df.columns else None)
    def normalize_edu(v: str) -> str:
        t = str(v).strip()
        low = t.lower().replace("degree","").replace(".","").replace(" ", "").replace("'", "")
        if "bachelor" in low: return "Bachelor's"
        if "master"  in low: return "Master's"
        if "phd" in low or "doutor" in low: return "PhD"
        return t
    if col: df[col] = df[col].apply(normalize_edu)
    return df

df = prep_df(df)
print("Colunas:", list(df.columns))
for c in ['Education_Level','education_level']:
    if c in df.columns:
        print(c, "→", sorted(df[c].unique().tolist()))
display(df.head())

# ---------------------- Target & Features ----------------------
def guess_target(cols):
    cands = [c for c in cols if re.search(r"salary|remun|pay|wage", str(c), re.I)]
    for c in cands:
        if re.search(r"usd|annual|year", str(c), re.I): return c
    return cands[0] if cands else cols[0]

TARGET = guess_target(list(df.columns))
FEATURES = [c for c in df.columns if c != TARGET]
print("TARGET =", TARGET)
print("FEATURES =", FEATURES)

# ---------------------- EDA (auto) ----------------------
def find_col_like(patterns, prefer_numeric=True):
    for c in df.columns:
        lc = c.lower()
        if any(p in lc for p in patterns):
            if not prefer_numeric or pd.api.types.is_numeric_dtype(df[c]):
                return c
    return None

def trendline(ax, x, y):
    m, b = np.polyfit(x, y, 1)
    xx = np.linspace(x.min(), x.max(), 100)
    yy = m*xx + b
    ax.plot(xx, yy)

plt.figure(); plt.hist(df[TARGET].dropna(), bins=30); plt.title(f"Distribuição do alvo: {TARGET}"); plt.xlabel(TARGET); plt.ylabel("Frequência"); plt.show()
plt.figure(); plt.boxplot(df[TARGET].dropna(), labels=[TARGET]); plt.title(f"Boxplot do alvo: {TARGET}"); plt.show()

cat_feats = [c for c in FEATURES if not pd.api.types.is_numeric_dtype(df[c])]
for c in cat_feats[:2]:
    top = df[c].value_counts().head(10).index
    sub = df[df[c].isin(top)]
    means = sub.groupby(c)[TARGET].mean().sort_values(ascending=False)
    plt.figure(); means.plot(kind='bar'); plt.title(f"Média de {TARGET} por {c} (Top 10)"); plt.xlabel(c); plt.ylabel(f"Média de {TARGET}")
    plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
if len(num_cols) >= 2:
    corr = df[num_cols].corr()
    fig, ax = plt.subplots()
    im = ax.imshow(corr, aspect='auto'); fig.colorbar(im, fraction=0.046, pad=0.04)
    ax.set_title("Matriz de correlação (com valores)")
    ax.set_xticks(range(len(num_cols))); ax.set_yticks(range(len(num_cols)))
    ax.set_xticklabels(num_cols, rotation=90); ax.set_yticklabels(num_cols)
    for i in range(len(num_cols)):
        for j in range(len(num_cols)):
            ax.text(j, i, f"{corr.iloc[i,j]:.2f}", ha='center', va='center', fontsize=8)
    plt.tight_layout(); plt.show()

age_col = find_col_like(['age'])
exp_col = find_col_like(['experience','years','yoe','exp'])
if age_col and exp_col and pd.api.types.is_numeric_dtype(df[age_col]) and pd.api.types.is_numeric_dtype(df[exp_col]):
    fig, ax = plt.subplots()
    ax.scatter(df[age_col], df[exp_col])
    trendline(ax, df[age_col].values, df[exp_col].values)
    ax.set_title(f"{age_col} × {exp_col} (com linha de tendência)"); ax.set_xlabel(age_col); ax.set_ylabel(exp_col)
    plt.tight_layout(); plt.show()

if exp_col and pd.api.types.is_numeric_dtype(df[exp_col]):
    fig, ax = plt.subplots()
    ax.scatter(df[exp_col], df[TARGET])
    trendline(ax, df[exp_col].values, df[TARGET].values)
    ax.set_title(f"{exp_col} × {TARGET} (com linha de tendência)"); ax.set_xlabel(exp_col); ax.set_ylabel(TARGET)
    plt.tight_layout(); plt.show()

# ---------------------- Modeling (auto) ----------------------
data = df.dropna(subset=[TARGET]).copy()
X = data[FEATURES].copy()
y = data[TARGET].astype(float)

num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]

prep = ColumnTransformer([('num', StandardScaler(), num_cols),
                          ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(random_state=42, n_estimators=300),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

results = []
for name, model in models.items():
    pipe = Pipeline([('prep', prep), ('model', model)])
    pipe.fit(Xtr, ytr)
    p = pipe.predict(Xte)
    r2 = r2_score(yte, p)
    mse = mean_squared_error(yte, p)
    rmse = float(np.sqrt(mse))
    mae = mean_absolute_error(yte, p)
    results.append((name, pipe, r2, rmse, mae))

grid = GridSearchCV(Pipeline([('prep', prep), ('model', GradientBoostingRegressor(random_state=42))]),
                    {'model__n_estimators':[100,200], 'model__learning_rate':[0.05,0.1], 'model__max_depth':[2,3]},
                    scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
grid.fit(Xtr, ytr)
p = grid.best_estimator_.predict(Xte)
r2 = r2_score(yte, p); rmse = float(np.sqrt(mean_squared_error(yte, p))); mae = mean_absolute_error(yte, p)
results.append(("GradientBoosting(GridSearch)", grid.best_estimator_, r2, rmse, mae))

results_sorted = sorted(results, key=lambda x: x[3])
print("Resultados (ordenado por RMSE):")
for n, _, r2, rmse, mae in results_sorted:
    print(f"{n:30s}  R²={r2:.3f}  RMSE={rmse:.3f}  MAE={mae:.3f}")

labels = [r[0] for r in results_sorted]
r2_vals = [r[2] for r in results_sorted]
rmse_vals = [r[3] for r in results_sorted]
mae_vals = [r[4] for r in results_sorted]

plt.figure(); plt.bar(labels, r2_vals); plt.title("R²"); plt.xticks(rotation=30, ha='right'); plt.tight_layout(); plt.show()
plt.figure(); plt.bar(labels, rmse_vals); plt.title("RMSE"); plt.xticks(rotation=30, ha='right'); plt.tight_layout(); plt.show()
plt.figure(); plt.bar(labels, mae_vals); plt.title("MAE"); plt.xticks(rotation=30, ha='right'); plt.tight_layout(); plt.show()

BEST_NAME, BEST_PIPE = results_sorted[0][0], results_sorted[0][1]
print("Melhor (RMSE):", BEST_NAME)

# ---------------------- Prediction UI (auto-update) ----------------------
def build_controls(features):
    ctrls = []
    for c in features:
        if pd.api.types.is_numeric_dtype(df[c]):
            lo, hi = float(df[c].quantile(0.05)), float(df[c].quantile(0.95))
            step = (hi-lo)/100 if hi>lo else 1.0
            if pd.api.types.is_float_dtype(df[c]):
                w = widgets.FloatSlider(description=c, min=lo, max=hi if hi>lo else lo+1, step=step, value=float(df[c].median()), continuous_update=True)
            else:
                w = widgets.IntSlider(description=c, min=int(lo), max=int(hi) if hi>lo else int(lo)+1, step=max(1,int(step)), value=int(df[c].median()), continuous_update=True)
        else:
            opts = sorted(df[c].astype(str).unique().tolist())
            w = widgets.Dropdown(description=c, options=opts, value=opts[0] if opts else None)
        ctrls.append(w)
    return ctrls

controls = build_controls(FEATURES)
box = widgets.VBox(controls)
out = widgets.Output()

def update_prediction(change=None):
    data = {c.description:[c.value] for c in controls}
    Xnew = pd.DataFrame(data)
    yhat = BEST_PIPE.predict(Xnew)[0]
    out.clear_output()
    with out:
        print(f"Previsão de salário: {yhat:,.2f}")

for c in controls:
    if hasattr(c, 'observe'):
        c.observe(update_prediction, names='value')

display(box, out)
update_prediction()

# ---------------------- Optional: Confusion Matrix (didactic) ----------------------
y_cont = data[TARGET].astype(float).values
bins = np.quantile(y_cont, np.linspace(0,1,4))  # 3 bins
bins[0] -= 1e-6; bins[-1] += 1e-6
y_cat = np.digitize(y_cont, bins[1:-1])
Xc = data[FEATURES].copy()

num_cols = [c for c in Xc.columns if pd.api.types.is_numeric_dtype(Xc[c])]
cat_cols = [c for c in Xc.columns if c not in num_cols]
prep_c = ColumnTransformer([('num', StandardScaler(), num_cols), ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

Xtr, Xte, ytr, yte = train_test_split(Xc, y_cat, test_size=0.2, random_state=42)
clf = Pipeline([('prep', prep_c), ('model', RandomForestClassifier(random_state=42, n_estimators=300))])
clf.fit(Xtr, ytr)
yp = clf.predict(Xte)

cm = confusion_matrix(yte, yp, labels=[0,1,2])
fig, ax = plt.subplots()
im = ax.imshow(cm, cmap='Blues')
ax.set_title("Matriz de confusão (bins do target)"); ax.set_xlabel("Predito"); ax.set_ylabel("Verdadeiro")
ax.set_xticks([0,1,2]); ax.set_yticks([0,1,2])
for i in range(3):
    for j in range(3):
        ax.text(j, i, str(cm[i,j]), ha='center', va='center')
fig.colorbar(im, fraction=0.046, pad=0.04)
plt.tight_layout(); plt.show()
